<a href="https://colab.research.google.com/github/vikaspathak0911/Python/blob/main/requirements_generator_of_python_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ast
import os
import argparse
from pathlib import Path

def find_main_file(project_path: Path):
    """
    Uses heuristics to find the most likely main entry point of a project.
    """
    # 1. Check for common, explicit names in the root directory.
    for name in ['main.py', 'app.py', 'run.py']:
        if (project_path / name).exists():
            return project_path / name

    # 2. Check for manage.py, which is a clear indicator of a Django project.
    if (project_path / 'manage.py').exists():
        return project_path / 'manage.py'

    # 3. If no obvious file is found, parse all .py files to find
    #    one containing an `if __name__ == "__main__":` block.
    main_candidates = []
    for py_file in project_path.rglob('*.py'):
        # Avoid searching inside virtual environment folders
        if 'venv' in py_file.parts or '.venv' in py_file.parts:
            continue
        try:
            with open(py_file, 'r', encoding='utf-8') as f:
                content = f.read()
            tree = ast.parse(content, filename=py_file.name)
            for node in ast.walk(tree):
                if isinstance(node, ast.If):
                    # Robustly check for `if __name__ == "__main__":`
                    test = node.test
                    if (isinstance(test, ast.Compare) and
                        isinstance(test.left, ast.Name) and
                        test.left.id == '__name__' and
                        len(test.ops) == 1 and isinstance(test.ops[0], ast.Eq) and
                        len(test.comparators) == 1 and
                        isinstance(test.comparators[0], ast.Constant) and
                        test.comparators[0].value == '__main__'):
                        main_candidates.append(py_file)
                        break  # Found it in this file, move to the next
        except (SyntaxError, UnicodeDecodeError, OSError):
            # Ignore files that cannot be parsed
            continue

    if main_candidates:
        # Prefer candidates in the root directory over nested ones.
        root_candidates = [f for f in main_candidates if f.parent == project_path]
        if root_candidates:
            return root_candidates[0]
        return main_candidates[0] # Otherwise, just return the first one found.

    return None



In [ ]:
def detect_framework(project_path: Path):
    """
    Performs a simple scan to detect common web frameworks.
    """
    if (project_path / 'manage.py').exists():
        return 'Django'

    for py_file in project_path.rglob('*.py'):
        try:
            with open(py_file, 'r', encoding='utf-8') as f:
                content = f.read()
            if 'from flask import Flask' in content:
                return 'Flask'
            if 'from fastapi import FastAPI' in content:
                return 'FastAPI'
        except (UnicodeDecodeError, OSError):
            continue
    return 'Unknown'




In [ ]:
def generate_run_instructions(folder_path, output_file='HOW_TO_RUN.md'):
    """
    Analyzes a project folder and generates a markdown file with setup and run instructions.
    """
    project_path = Path(folder_path).resolve()
    print(f"Analyzing project at: {project_path}")
    if not project_path.is_dir():
        print(f"Error: '{folder_path}' is not a valid directory.")
        return

    instructions = [
        "# How to Run This Project",
        f"These are auto-generated instructions for running the project located at `{project_path}`.",
        "Please review and adapt them as necessary, as complex projects may require additional setup."
    ]

    # --- Step 1: Virtual Environment ---
    instructions.extend([
        "\n## Step 1: Set Up a Virtual Environment",
        "It is a best practice to use a virtual environment to manage project dependencies and avoid conflicts.",
        "```bash",
        "# Navigate to your project directory",
        f"cd \"{project_path}\"",
        "\n# Create a virtual environment (e.g., named 'venv')",
        "python -m venv venv",
        "\n# Activate the virtual environment",
        "# On Windows (Command Prompt/PowerShell):",
        "venv\\Scripts\\activate",
        "# On macOS and Linux:",
        "source venv/bin/activate",
        "```"
    ])

    # --- Step 2: Install Dependencies ---
    instructions.extend(["\n## Step 2: Install Dependencies"])
    if (project_path / 'requirements.txt').exists():
        instructions.extend([
            "A `requirements.txt` file was found. Install the required packages using pip:",
            "```bash",
            "pip install -r requirements.txt",
            "```"
        ])
    else:
        instructions.extend([
            "No `requirements.txt` file was found.",
            "You may need to create one. You can use the `generate_requirements.py` script to scan the project and create this file, or use a tool like `pipreqs`.",
            "```bash",
            "# Example using pipreqs (you may need to install it first: pip install pipreqs)",
            "pipreqs .",
            "\n# Then install the dependencies",
            "pip install -r requirements.txt",
            "```"
        ])

    # --- Step 3: Run the Project ---
    instructions.extend(["\n## Step 3: Run the Application"])
    main_file = find_main_file(project_path)
    framework = detect_framework(project_path)

    run_command_found = False
    if framework == 'Django':
        instructions.extend([
            "This appears to be a **Django** project. The standard way to run the development server is:",
            "```bash",
            "python manage.py runserver",
            "```"
        ])
        run_command_found = True
    elif main_file:
        relative_main_file = main_file.relative_to(project_path)
        if framework == 'FastAPI':
            instructions.extend([
                "This appears to be a **FastAPI** project. You'll likely need `uvicorn` to run it.",
                "If `uvicorn` is not in your requirements, install it with `pip install \"uvicorn[standard]\"`.",
                f"The run command is typically `uvicorn <filename>:<app_variable> --reload`. Based on the file `{relative_main_file}`, a likely command is:",
                "```bash",
                f"uvicorn {relative_main_file.stem}:app --reload",
                "```",
                f"(You may need to replace `app` with the actual name of your FastAPI application instance in `{relative_main_file}`)."
            ])
            run_command_found = True
        else: # Generic Python or Flask
            if framework == 'Flask':
                 instructions.append("This appears to be a **Flask** project.")
            instructions.extend([
                f"The likely entry point for this project is `{relative_main_file}`. You can probably run it with:",
                "```bash",
                f"python {relative_main_file}",
                "```"
            ])
            run_command_found = True

    if not run_command_found:
        instructions.extend([
            "Could not automatically determine the main file or run command.",
            "Please inspect the source code to find the main entry point. Look for a file named `main.py`, `app.py`, or a file containing `if __name__ == '__main__':`."
        ])

    # --- Step 4: Additional Notes ---
    instructions.extend(["\n## Additional Notes"])
    notes_found = False
    if (project_path / 'README.md').exists():
        instructions.append("- A `README.md` file was found. **It is highly recommended to read it** for project-specific instructions and details.")
        notes_found = True
    if list(project_path.glob('.env*')) or list(project_path.glob('config.py.example')):
        instructions.append("- An example configuration or environment file was found (e.g., `.env.example`). You may need to copy it (e.g., `cp .env.example .env`) and fill in the required values like API keys or database URLs.")
        notes_found = True

    if not notes_found:
        instructions.append("- No common additional configuration files (like `README.md` or `.env.example`) were found.")


    # --- Write to file ---
    output_path = Path(output_file)
    with output_path.open('w', encoding='utf-8') as f:
        f.write("\n".join(instructions))

    print(f"\nSuccessfully generated instructions in '{output_path.resolve()}'")



In [ ]:
def main():
    """Main function to handle command-line arguments."""
    parser = argparse.ArgumentParser(
        description="Analyzes a Python project and generates a HOW_TO_RUN.md file with step-by-step instructions.",
        formatter_class=argparse.RawTextHelpFormatter
    )
    parser.add_argument(
        "folder",
        nargs='?',
        default='.',
        help="The path to the folder containing your Python project (default: current directory)."
    )
    parser.add_argument(
        "-o", "--output",
        default="HOW_TO_RUN.md",
        help="The name of the output instructions file (default: HOW_TO_RUN.md)."
    )

    args = parser.parse_args()
    generate_run_instructions(args.folder, args.output)


In [ ]:

if __name__ == "__main__":
    main()
